In [1]:
# built-ins
import sys
import os
sys.path.append(os.path.abspath('..'))

# modules
from container_maker_spec.service_pb2_grpc import ContainerMakerAPIStub
from container_maker_spec.types_pb2 import CreateContainerRequest
from container_maker_spec.types_pb2 import GetContainerRequest
from container_maker_spec.types_pb2 import ListContainerRequest
from container_maker_spec.types_pb2 import DeleteContainerRequest
from container_maker_spec.types_pb2 import ContainerResponse
from src.common.utils import read_certs

# third party
import grpc

## Channels
The first step is to create GRPC channels. We have two types of channels, `insecure` and `secure`.
Based on the environment choose the right channel.

There are two cells, choose one of them.

In [4]:
# insecure channel
ic: grpc.Channel = grpc.insecure_channel("container-maker-development-service:50052")
stub: ContainerMakerAPIStub = ContainerMakerAPIStub(channel=ic)

In [5]:
# secure channel
client_key: bytes = read_certs('CLIENT_KEY', './cert/client.key')
client_cert: bytes = read_certs('CLIENT_CRT', './cert/client.crt')
ca_cert: bytes = read_certs('CA_CRT', './cert/ca.crt')

credentials: grpc.ServerCredentials = grpc.ssl_channel_credentials(
    root_certificates=ca_cert,
    private_key=client_key,
    certificate_chain=client_cert
)

sc: grpc.Channel = grpc.secure_channel("container-maker-development-service:50052", credentials)
stub: ContainerMakerAPIStub = ContainerMakerAPIStub(channel=sc)


## Operations
These are the operations:
1. Create Container
2. Get Container
3. List Containers
4. Delete Container

Below we have examples for each

In [6]:
# Create Container
from container_maker_spec.types_pb2 import ExposureLevel as GRPCExposureLevel

create_container_request: CreateContainerRequest = CreateContainerRequest(
    container_name='test-ssh-container',
    network_name='test-ssh-namespace',
    image_name='zim95/ssh_ubuntu:latest',
    exposure_level=GRPCExposureLevel.EXPOSURE_LEVEL_CLUSTER_LOCAL,
    publish_information=[
        {'publish_port': 2222, 'target_port': 22, 'protocol': 'TCP'}
    ],
    environment_variables={
        'SSH_PASSWORD': '12345678',
        'SSH_USERNAME': 'test-user',
    }
)

create_container_response = stub.createContainer(create_container_request)
print(create_container_response)

container_id: "5596e2c8-942d-4c3b-80b4-d711ac0ffc6d"
container_name: "test-ssh-container-service"
container_ip: "10.106.109.52"
container_network: "test-ssh-namespace"
ports {
  container_port: 2222
  protocol: "TCP"
}



In [9]:
# Get Container
get_container_request: GetContainerRequest = GetContainerRequest(
    container_id='5596e2c8-942d-4c3b-80b4-d711ac0ffc6d', network_name='test-ssh-namespace'
)
get_container_response = stub.getContainer(get_container_request)
print(get_container_response)

container_id: "5596e2c8-942d-4c3b-80b4-d711ac0ffc6d"
container_name: "test-ssh-container-service"
container_ip: "10.106.109.52"
container_network: "test-ssh-namespace"
ports {
  container_port: 2222
  protocol: "TCP"
}



In [7]:
# list containers
list_container_request: ListContainerRequest = ListContainerRequest(network_name='test-ssh-namespace')
list_container_response = stub.listContainer(list_container_request)
print(list_container_response)

containers {
  container_id: "5596e2c8-942d-4c3b-80b4-d711ac0ffc6d"
  container_name: "test-ssh-container-service"
  container_ip: "10.106.109.52"
  container_network: "test-ssh-namespace"
  ports {
    container_port: 2222
    protocol: "TCP"
  }
}



In [ ]:
# delete container
delete_container_request: DeleteContainerRequest = DeleteContainerRequest(
    container_id='09db196d-8503-48f6-bc16-5ee4e6cbc675', network_name='test-ssh-namespace'
)
delete_container_response = stub.deleteContainer(delete_container_request)
print(delete_container_response)